In [6]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

model = (AutoModelForCausalLM.from_pretrained
    (
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="auto",
    torch_dtype="auto",
    #attn_implementation = 'eager'
    )
)

rag_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype="auto",
    #attn_implementation = 'eager'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [7]:
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"

#lädt datenbank lokal runter
client = chromadb.PersistentClient(path=db_path)

#unser emdedding model
embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

#"meinungen" aus der Datenbank laden
collection = client.get_collection("meinungen")

#retrievet daten von unserer Vektorendatenbank
def query_collection(query_text, n_results=4):

    #encode query text (-> unser input)
    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

In [8]:
#Funktion, die die Datenbank abfragt und eine Antwort generiert
def query_with_phi3_5(query_text):
    results = query_collection(query_text)
    context = ""
    for i in range(len(results["documents"][0])):
        context += f"Document {i+1}:\n"
        context += f"{results['documents'][0][i]}\n"
        context += f"Party: {results['metadatas'][0][i]['party']}\n"


    prompt = f"""
    You are a political AI assistant.  Based on the following information retrieved from a vector database, answer the user's question.  If the provided data is not relevant to the question, respond with "I do not know".

    User Question: {query_text}

    Retrieved Data:
    {context}

    Your Answer:
    """

    generated_text = rag_pipeline(prompt, max_new_tokens=250)[0]["generated_text"]
    # Extract the answer from the generated text (remove the prompt)
    answer = generated_text[len(prompt):].strip()
    return answer


query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_phi3_5(query)
print(answer)

Die AfD sieht den Islamunterricht an deutschen Schulen kritisch und befürwortet eine sachliche Islamkunde im Ethikunterricht. Sie befürchten, dass der Islamunterricht echte Integration verhindern könnte, da islamische Gemeinschaften keine kirchenähnliche Struktur aufweisen und daher kein „bekenntnisgebundener“ Religionsunterricht zugestanden werden kann. Die Partei fordert, dass Imame und Lehrer*innen in deutscher Sprache und mit einem Zertifikat B2 für die deutsche Sprache des Gemeinsamen Europäischen Referenzrahmens für Sprachen zugelassen werden. Zudem lehnt die AfD die Verleihung des Status als Körperschaft öffentlichen Rechts an islamische Organisationen ab und fordert die Abschaffung der islamtheologischen Lehrstühle an deutschen Universitäten. Sie unterstützt zudem die Gleichberechtigung von Mann und Frau und fordert die Untersagung des Tragens von Burka und Niqab in der Öffentlichkeit.


Document:


In [6]:
from rouge_score import rouge_scorer

# Funktion zur Berechnung der ROUGE-Metrik
def evaluate_rouge(predictions, references):
    # Definiere die zu berechnenden ROUGE-Metriken
    rouge_types = ["rouge1", "rouge2", "rougeL"]
    
    # Initialisiere den scorer mit den gewünschten Metriken
    scorer = rouge_scorer.RougeScorer(rouge_types=rouge_types)
    
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}
    
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        scores["rouge1"].append(score["rouge1"].fmeasure)
        scores["rouge2"].append(score["rouge2"].fmeasure)
        scores["rougeL"].append(score["rougeL"].fmeasure)

    # Durchschnittliche F1-Score für jede ROUGE-Metrik berechnen
    avg_scores = {metric: sum(scores[metric])/len(scores[metric]) for metric in scores}
    return avg_scores

# Beispiel für eine generierte Antwort und eine Referenzantwort
generated_answer = "Die AFD ist kritisch gegenüber dem Islamunterricht in deutschen Schulen."
reference_answer = "Die AFD lehnt den Islamunterricht an deutschen Schulen ab und betrachtet ihn als problematisch."

# Aufruf der Evaluierungsfunktion
average_scores = evaluate_rouge([generated_answer], [reference_answer])

# Ausgabe der Ergebnisse
print("Durchschnittliche ROUGE-Scores:")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")


Durchschnittliche ROUGE-Scores:
rouge1: 0.4000
rouge2: 0.1739
rougeL: 0.4000


In [7]:
from sklearn.metrics import accuracy_score, f1_score

y_true = [1]  # Wahre Label (z.B. Ja=1, Nein=0)
y_pred = [1]  # Vorhergesagte Label

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 1.0000
F1 Score: 1.0000


In [9]:
import spacy

# Lade ein NER-Modell
nlp = spacy.load("de_core_news_sm")

reference = "Die AFD lehnt den Islamunterricht an deutschen Schulen ab."
candidate = "Die AFD ist kritisch gegenüber dem Islamunterricht in deutschen Schulen."

reference_doc = nlp(reference)
candidate_doc = nlp(candidate)

reference_entities = set(ent.text for ent in reference_doc.ents)
candidate_entities = set(ent.text for ent in candidate_doc.ents)

print("Reference Entities:", reference_entities)
print("Candidate Entities:", candidate_entities)
print("Matching Entities:", reference_entities & candidate_entities)


Reference Entities: {'AFD', 'deutschen'}
Candidate Entities: {'AFD', 'deutschen'}
Matching Entities: {'AFD', 'deutschen'}
